# Systemy uczące się - Zad. dom. 1: Minimalizacja ryzyka empirycznego
Celem zadania jest zaimplementowanie własnego drzewa decyzyjnego wykorzystującego idee minimalizacji ryzyka empirycznego. 

### Autor rozwiązania
Uzupełnij poniższe informacje umieszczając swoje imię i nazwisko oraz numer indeksu:

In [ ]:
NAME = "Weronika Koga"
ID = "151574"

## Twoja implementacja

Twoim celem jest uzupełnić poniższą klasę `TreeNode` tak by po wywołaniu `TreeNode.fit` tworzone było drzewo decyzyjne minimalizujące ryzyko empiryczne. Drzewo powinno wspierać problem klasyfikacji wieloklasowej (jak w przykładzie poniżej). Zaimplementowany algorytm nie musi (ale może) być analogiczny do zaprezentowanego na zajęciach algorytmu dla klasyfikacji. Wszelkie przejawy inwencji twórczej wskazane. **Pozostaw komenatrze w kodzie, które wyjaśniają Twoje rozwiązanie.**

Schemat oceniania:
- wynik na zbiorze Iris (automatyczna ewaluacja) celność klasyfikacji >= prostego baseline'u + 10%: +40%,
- wynik na ukrytym zbiorze testowym 1 (automatyczna ewaluacja) celność klasyfikacji >= prostego baseline'u + 15%: +30%,
- wynik na ukrytym zbiorze testowym 2 (automatyczna ewaluacja) celność klasyfikacji >= prostego baseline'u + 5%: +30%.

Niedozwolone jest korzystanie z zewnętrznych bibliotek do tworzenia drzewa decyzyjnego (np. scikit-learn). 
Możesz jedynie korzystać z biblioteki numpy.

#### Uwaga: Możesz dowolnie modyfikować elementy tego notebooka (wstawiać komórki i zmieniać kod), o ile będzie się w nim na koniec znajdowała kompletna implementacja klasy `TreeNode` w jednej komórce.

In [8]:
import numpy as np


class TreeNode:

    def __init__(self):

        self.left: TreeNode | None = None  # wierzchołek znajdujący się po lewej stornie
        self.right: TreeNode | None = (
            None  # wierzchołek znajdujący się po prawej stornie
        )
        self.value: int | None = None  # wartość w wierzchołku (tylko dla liści)
        self.feature: int | None = None  # indeks cechy, według której dokonano podziału
        self.threshold: float | None = (
            None  # wartość progu, według którego dokonano podziału
        )

    def _entropy(self, y):
        hist = np.bincount(
            y
        )  # super funkcja z np do zliczania liczb wystąpień każdej klasy
        ps = hist / len(y)  # prawdopodobieństwa wystąpień każdej klasy
        return -np.sum(
            [p * np.log(p) for p in ps if p > 0]
        )  # wzór na entropię. list comprehension oblicza dla każdego prawdopodobieństwa p * log(p) ostatni warunek zapobiega logarytmowi z 0

    def _information_gain(self, y, left_idxs, right_idxs):
        # entropia rodzica
        parent_entropy = self._entropy(y)

        # entropia dzieci
        n_samples = len(y)
        n_left_samples, num_right_samples = len(left_idxs), len(right_idxs)
        entropy_left, entropy_right = self._entropy(y[left_idxs]), self._entropy(
            y[right_idxs]
        )
        child_entropy = (n_left_samples / n_samples) * entropy_left + (
            num_right_samples / n_samples
        ) * entropy_right  # średnia ważona entropia dzieci

        # licze information gain wg wzoru
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(
            X_column <= split_thresh
        ).flatten()  # flatten zeby miec ladna plaska liste, wszystkich indeksow mniejszych od progu
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def fit(self, data: np.ndarray, target: np.ndarray) -> None:
        """

        Args:

                data (np.ndarray): macierz cech o wymiarach (n, m), gdzie n to liczba przykładów, a m to liczba cech

                target (np.ndarray): wektor klas o długości n, gdzie n to liczba przykładów


        """
        best_information_gain = 0
        best_feature = None
        best_threshold = None
        best_left_indices = None
        best_right_indices = None

        current_entropy = self._entropy(target)

        # Jeśli entropia = 0 to znaczy, że wszystkie próbki mają tę samą klasę, więc target[0] czy target[whatever] nie ma znaczenia.
        if current_entropy == 0:
            self.value = target[0]
            return

        # Iteruj przez każdą cechę
        for feature_index in range(data.shape[1]):
            # Pobierz unikalne wartości dla danej cechy
            thresholds = np.unique(data[:, feature_index])

            # Iteruj przez unikalne wartości cechy jako progi
            for threshold in thresholds:

                left_indices, right_indices = self._split(
                    data[:, feature_index], threshold
                )
                # To sie moze zdarzyc jak wszystkie wartosci sa takie same. Wtedy nie bede liczyc IG
                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue
                else:
                    information_gain = self._information_gain(
                        target, left_indices, right_indices
                    )

                # aktualny podzial okazuje sie najlepszy, wiec dostosowuje wszystkie zmienne tak abym mogla go wykorzystac
                if information_gain > best_information_gain:
                    best_information_gain = information_gain
                    best_feature = feature_index
                    best_threshold = threshold
                    best_left_indices, best_right_indices = left_indices, right_indices

        # 	obecny Node jest liściem, wiec zapisz jego odpowiedź
        if best_information_gain == 0:
            self.value = np.argmax(
                np.bincount(target)
            )  # value to najczęściej występująca klasa w target
            return

        else:
            self.feature = best_feature
            self.threshold = best_threshold
            self.left = TreeNode()
            self.right = TreeNode()

            self.right.fit(data[best_right_indices], target[best_right_indices])
            self.left.fit(data[best_left_indices], target[best_left_indices])

    def predict(self, data: np.ndarray) -> np.ndarray:
        """

        Args:

                data (np.ndarray): macierz cech o wymiarach (n, m), gdzie n to liczba przykładów, a m to liczba cech


        Returns:

                np.ndarray: wektor przewidzoanych klas o długości n, gdzie n to liczba przykładów

        """

        y_pred = np.zeros(data.shape[0])

        for i, sample in enumerate(data):

            node = self  # tworze korzeń drzewa

            while node.value is None:  # dopóki nie jestem w liściu

                if sample[node.feature] > node.threshold:

                    node = node.right

                else:

                    node = node.left

            y_pred[i] = (
                node.value
            )  # przypisuje finalna wartość z liścia jako moja predykcje dla danego przykładu

        return y_pred

## Przykład trenowanie i testowania drzewa
 
Później znajduje się przykład trenowania i testowania drzewa na zbiorze danych `iris`, który zawierający 150 próbek irysów, z czego każda próbka zawiera 4 atrybuty: długość i szerokość płatków oraz długość i szerokość działki kielicha. Każda próbka należy do jednej z trzech klas: `setosa`, `versicolor` lub `virginica`, które są zakodowane jak int.

Możesz go wykorzystać do testowania swojej implementacji. Możesz też zaimplementować własne testy lub użyć innych zbiorów danych, np. innych [zbiorów danych z scikit-learn](https://scikit-learn.org/stable/datasets/toy_dataset.html#toy-datasets).

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.33, random_state=2024
)

tree_model = TreeNode()
tree_model.fit(X_train, y_train)
y_pred = tree_model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.96


# Uwagi

Robię drzewo decyzyjne po raz pierwszy w życiu w tym momencie. Myślałam o tym, żeby dodać może jakieś hiperparametry np. max_depth albo min_leaf_to_split. Konkluzja z tego taka, że i tak nie wiem, na jakim zbiorze danych Pan Doktor będzie testował model. Używając random_state=2024, uzyskałam wynik 0.84. Rozmawiałam ze znajomymi i niektórzy z nich również otrzymali podobne wyniki na zbiorze testowym, co pozwoliło im uzyskać ocenę 5.0. Próbowałam różnych wartości random_state i wyniki były zazwyczaj nie gorsze niż 0.82, a najlepszy wynik wynosił 0.96 gdy random state = 2023. 

Stwierdziłam, że chyba nie ma co dostosowywać hiperparametrów skoro całość i tak będzie oceniana na innym datasecie (chyba że czegoś nie zrozumiałam) a inny dataset będzie potrzebować innego tunningu (no chyba, że miałabym zaimplementować jakiś grid search, ale używając samego numpy to powoli robi się overkill). Jeśli uważa Pan, że na 5.0 to za mało, mogę poprawić zadanie, dodać jakieś lepsze warunki stopu i ten tunning hiperparametrow.